# Workflow CAVE + HpBandster

We will present you a example workflow of how to efficiently optimize a algorithm using our frameworks 
<a href="https://github.com/automl/CAVE" target="_blank">CAVE</a> and <a href="https://github.com/automl/HpBandSter" target="_blank">HpBandSter</a>

## Short introduction to the used frameworks
### CAVE

Hier sollte eine kurze Beschreibung von Cave stehen.

### HpBandSter

Modern deep learning methods are very sensitive to many hyperparameters, and, due to the long training times of state-of-the-art models, vanilla Bayesian hyperparameter optimization is typically computationally infeasible. On the other hand, bandit-based configuration evaluation approaches based on random search lack guidance and do not converge to the best configurations as quickly. With HpBanster, we propose to combine the benefits of both Bayesian optimization and bandit-based methods, in order to achieve the best of both worlds: strong anytime performance and fast convergence to optimal configurations. We propose a new practical state-of-the-art hyperparameter optimization method, which consistently outperforms both Bayesian optimization and Hyperband on a wide range of problem types, including high-dimensional toy functions, support vector machines, feed-forward neural networks, Bayesian neural networks, deep reinforcement learning, and convolutional neural networks. Our method is robust and versatile, while at the same time being conceptually simple and easy to implement.

For more insights, please consider the paper: <a href="https://arxiv.org/abs/1807.01774" target="_blank">BOHB: Robust and Efficient Hyperparameter Optimization at Scale</a>

## The Workflow


In the next section, we guide you through the following steps:
#### 1) Given a algorithm to optimize and a *configuration space*, we will run BOHB on this problem. 
This step contains: 
1. Setting up a worker, which runs the given algorithmn with all the sampled configurations. Here it's a simple scipy implementation of a svm, training to classify the MNISTdataset.
2. Defining a configurations space for the classifier using our <a href="https://github.com/automl/ConfigSpace" target="_blank">ConfigSpace module</a>.
3. Setting up a nameserver, which organizes the possible multiple workers
4. Starting the optimizer, here BOHB.


This will return us the optimization run results. For example the best hyperparameter configuration, which is often referred to as *incumbent*. 
Also a lot of information like which configurations has been used, as well as their performances.
#### 2) We will pass the BOHB results into the CAVE-tool.
It will give insights into the 
1. Parameter importance, 
2. performance analysis,
3. feature analysis and 
4. configuration behaviour.

### 1.1) Setting up a worker class
+ inherit from the hpbandster.core.worker class
+ load the mnist data in the init-method
+ overwrite the compute methode: the training of the model happens here
+ make sure that the returned dictionary contains the fields **loss** and **info**

In [96]:
from sklearn import datasets, neural_network, metrics
from hpbandster.core.worker import Worker

class MyWorker(Worker):
    def __init__(self):
        # We are loading the MNIST dataset and split it into a
        # training and a test set.
        digits = datasets.load_digits()
        n_samples = len(digits.images)
        data = digits.images.reshape((n_samples, -1))
        
        self.train_x = data[:n_samples // 2]
        self.train_y = digits.target[:n_samples // 2]
        self.test_x = data[n_samples // 2:]
        self.test_y = digits.target[n_samples // 2:]

    def compute(self, config, budget, *args, **kwargs):
        """
        Simple example for a compute function. It'll be repeatedly called by the optimizer. 
        
        Args:
            config: dictionary containing the sampled configurations by the optimizer
            budget: (float) amount of time/epochs/etc. the model can use to train

        Returns:
            dictionary with mandatory fields:
                'loss' (scalar)
                'info' (dict)
        """
        beta_1 = 0  if 'beta_1' not in config else config['beta_1']
        beta_2 = 0  if 'beta_2' not in config else config['beta_2']
        
        clf = neural_network.MLPClassifier(max_iter=budget,
                                           learning_rate='constant',
                                           learning_rate_init=config['learning_rate_init'],
                                           activation=config['activation'],
                                           solver=config['solver'],
                                           beta_1=beta_1, 
                                           beta_2=beta_2
                                          )
        clf.fit(self.train_x, self.train_y)
        
        predicted = clf.predict(self.test_x)
        loss_train = metrics.log_loss(self.train_y, clf.predict_proba(self.train_x))
        loss_test = metrics.log_loss(self.test_y, clf.predict_proba(self.test_x))

        accuracy_train = clf.score(self.train_x, self.train_y)
        accuracy_test = clf.score(self.test_x, self.test_y)
        
        # print(clf.n_iter_)
                
        # print("Classification report for classifier %s:\n%s\n"
        #       % (classifier, metrics.classification_report(self.test_y, predicted)))
        # print("Confusion matrix:\n%s" % metrics.confusion_matrix(self.test_y, predicted))
        
        return ({
            'loss': loss_train,  # this is the a mandatory field to run hyperband
            'info': {'loss_train': loss_train,
                     'loss_test': loss_test,
                     'accuracy_train': accuracy_train,
                     'accuracy_test': accuracy_test,
                     
                    }  # can be used for any user-defined information - also mandatory
        })

In [102]:
m = MyWorker()
cs = sample_configspace()
cfg = cs.sample_configuration()
print(cfg)
m.compute(config=cfg, budget=50)

Configuration:
  activation, Value: 'tanh'
  learning_rate_init, Value: 0.007002728767685752
  solver, Value: 'sgd'



C:\Users\Philipp\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'info': {'accuracy_test': 0.9098998887652948,
  'accuracy_train': 1.0,
  'loss_test': 0.27973529987623924,
  'loss_train': 0.04369264158913515},
 'loss': 0.04369264158913515}

### 1.2) ConfigSpace:
Now we define the configuration space with the hyperparameters we'd like to optimize.

We import also the ConfigSpace-to-json-writer, to save the configuration space to file, which we will use later in CAVE.

In [94]:
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
import ConfigSpace.read_and_write.json as json_writer

def sample_configspace():
    config_space = CS.ConfigurationSpace()
    config_space.add_hyperparameter(CSH.CategoricalHyperparameter('activation', ['tanh', 'relu']))
    config_space.add_hyperparameter(CS.UniformFloatHyperparameter('learning_rate_init', lower=1e-6, upper=1e-2, log=True))
    
    solver = CSH.CategoricalHyperparameter('solver', ['sgd', 'adam'])
    config_space.add_hyperparameter(solver)
    
    beta_1 = CS.UniformFloatHyperparameter('beta_1', lower=0, upper=1)
    config_space.add_hyperparameter(beta_1)
    
    condition = CS.EqualsCondition(beta_1, solver, 'adam')
    config_space.add_condition(condition)
    
    beta_2 = CS.UniformFloatHyperparameter('beta_2', lower=0, upper=1)
    config_space.add_hyperparameter(beta_2)
    
    condition = CS.EqualsCondition(beta_2, solver, 'adam')
    config_space.add_condition(condition)
    
    return config_space

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

import ConfigSpace.read_and_write.json as json_writer
import hpbandster.core.nameserver as hpns

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.examples.commons import sample_configspace


# First, create a ConfigSpace-Object.
# It contains the hyperparameters to be optimized
# For more details, please have a look in the ConfigSpace-Example in the Documentation
config_space = sample_configspace()

# Write the ConfigSpace for later use to file
with open('example1_configspace.json', 'w') as fh:
    fh.write(json_writer.write(config_space))
v

# Every run has to have a unique (at runtime) id.
# This needs to be unique for concurrent runs, i.e. when multiple
# instances run at the same time, they have to have different ids
run_id = '0'


# Step 1:
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with a random port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
NS = hpns.NameServer(  run_id=run_id,
                       host='localhost',
                       port=0,
                    )
ns_host, ns_port = NS.start()


# Step 2:
# The worker implements the connection to the model to be evaluated.
# Its 'compute'-method will be called later by the BOHB-optimizer repeatedly
# with the sampled configurations and return the computed loss (and additional infos).
# Further usages of the worker will be covered in a later example.
w = MyWorker(   nameserver=ns_host,
                nameserver_port=ns_port,
                run_id=run_id,  # unique Hyperband run id (same as nameserver's)
            )
w.run(background=True)


# Step 3:
# In the last of the 3 Steps, we create an optimizer object.
# It samples configurations from the ConfigurationSpace, using succesive halfing.
# The number of sampled configurations is determined by the
# parameters eta, min_budget and max_budget.
# After evaluating each configuration, starting with the minimum budget
# on the same subset size, only a fraction of 1 / eta of them
# 'advances' to the next round. At the same time the current budget will be doubled.
# This process runs until the maximum budget is reached.
bohb = BOHB(  configspace = config_space,
              run_id = run_id,                       # same as nameserver's
              eta=3, min_budget=27, max_budget=243,  # Hyperband parameters
              nameserver=ns_host,
              nameserver_port = ns_port,
              ping_interval=3600,                    # how often master pings for workers (in seconds)
           )

# Then start the optimizer. The n_iterations parameter specifies
# the number of iterations to be performed in this run
res = bohb.run(n_iterations=2)

# After the optimizer run, we shutdown the master.
bohb.shutdown(shutdown_workers=True)


# BOHB will return a result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the result-object, see its documentation.
id2config = res.get_id2config_mapping()
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))


# The incumbent trajectory is a dictionary with all the configuration IDs, the times the runs
# finished, their respective budgets, and corresponding losses.
# It's used to do meaningful plots of the optimization process.
incumbent_trajectory = res.get_incumbent_trajectory()

import matplotlib.pyplot as plt
plt.plot(incumbent_trajectory['times_finished'], incumbent_trajectory['losses'])
plt.xlabel('wall clock time [s]')
plt.ylabel('incumbent loss')
plt.show()